## Import Dataset

In [2]:
import pandas as pd

In [3]:
root = "/home/sihartist/Desktop/"

In [4]:
# Load the credit card transaction data
df = pd.read_csv('/home/sihartist/Desktop/fraud-detection/dataset/data_balanced.csv')
df = df.iloc[:,1:]

df.head()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,CASH_IN,CASH_OUT,DEBIT,PAYMENT,TRANSFER,isFlaggedFraud,isFraud
0,139,265803.35,0.00,0.00,751669.39,1017472.74,0,1,0,0,0,0,0
1,589,11278.28,0.00,0.00,220970.84,232249.12,0,1,0,0,0,0,0
2,284,16363.06,35966.16,19603.10,0.00,0.00,0,0,0,1,0,0,0
3,37,107223.31,20752629.71,20859853.02,804594.62,571711.22,1,0,0,0,0,0,0
4,287,3678.25,0.00,0.00,0.00,0.00,0,0,0,1,0,0,0


## Prepare Training And Testing Data

In [5]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

2023-03-01 14:00:28.911920: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
# Split the data into training and test sets
X = df.drop('isFraud', axis=1).values
y = df['isFraud'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Reshape the data for input into the CNN
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1]))
print(X_train.shape)


(13140, 12)


## Build & Train Model

In [7]:
# Build the deep CNN model
model = tf.keras.Sequential([
    tf.keras.layers.Conv1D(32, 2, activation='relu', input_shape=(X_train.shape[1], 1)),
    tf.keras.layers.MaxPool1D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid', name='isFraud')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

2023-03-01 14:00:31.982174: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-01 14:00:31.986239: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/10
411/411 [==============================] - 6s 15ms/step - loss: 0.3087 - accuracy: 0.8575
Epoch 2/10
411/411 [==============================] - 4s 9ms/step - loss: 0.2180 - accuracy: 0.8992
Epoch 3/10
411/411 [==============================] - 2s 6ms/step - loss: 0.1991 - accuracy: 0.9111
Epoch 4/10
411/411 [==============================] - 6s 16ms/step - loss: 0.1843 - accuracy: 0.9187
Epoch 5/10
411/411 [==============================] - 7s 16ms/step - loss: 0.1762 - accuracy: 0.9207
Epoch 6/10
411/411 [==============================] - 4s 10ms/step - loss: 0.1755 - accuracy: 0.9250
Epoch 7/10
411/411 [==============================] - 5s 13ms/step - loss: 0.1646 - accuracy: 0.9275
Epoch 8/10
411/411 [==============================] - 5s 12ms/step - loss: 0.1632 - accuracy: 0.9288
Epoch 9/10
411/411 [==============================] - 5s 11ms/step - loss: 0.1570 - accuracy: 0.9298
Epoch 10/10
411/411 [==============================] - 5s 12ms/step - loss: 0.1543 - accuracy

## Evaluate Model

In [8]:
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test Accuracy:', test_acc)

103/103 [==============================] - 0s 3ms/step - loss: 0.1586 - accuracy: 0.9385
Test Accuracy: 0.9385271072387695


## Save CNN Model To HDF5 Format

In [9]:
model.save("models/cnn.h5")

## Save CNN Model To Pickle Format

In [10]:
import pickle

filename = root + "fraud-detection/algo/models/cnn.pkl"
pickle.dump(model, open(filename, "wb"))

INFO:tensorflow:Assets written to: ram://b6ce0985-d21c-46a8-b7e1-2b1e7d1774a8/assets


INFO:tensorflow:Assets written to: ram://b6ce0985-d21c-46a8-b7e1-2b1e7d1774a8/assets


## Save CNN Model To ONNX Format

In [11]:
import tensorflow as tf
import tf2onnx
import onnx

loaded_model = tf.keras.models.load_model('models/cnn.h5')
onnx_model, _ = tf2onnx.convert.from_keras(loaded_model)
onnx.save(onnx_model, "models/cnn.onnx")
onnx.save(onnx_model, root + "SpringAPI/src/main/resources/models/cnn.onnx")


2023-03-01 14:01:23.186589: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2023-03-01 14:01:23.186743: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session
2023-03-01 14:01:23.233821: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2023-03-01 14:01:23.238666: I tensorflow/core/grappler/devices.cc:75] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2023-03-01 14:01:23.238791: I tensorflow/core/grappler/clusters/single_machine.cc:358] Starting new session


## Load Model And Test A Prediction

In [12]:
test=[139, 265803.35, 0.00, 0.00, 751669.39, 1017472.74, 0, 1, 0, 0, 0 ,0]
test1=[319, 1033878.67, 1033878.67, 0.0, 0.0, 0.0, 0, 0, 0, 0, 1, 0]

to_predict1 = np.array(test).reshape(1,12)
to_predict2 = np.array(test1).reshape(1,12)

loaded_model = tf.keras.models.load_model("models/cnn.h5")
result = loaded_model.predict(to_predict1)
print(result)

1/1 [==============================] - 0s 90ms/step
[[0.]]
